# Transfer Learning Using tfhub

In [ ]:
# install dependencies
!pip install -q tensorflow tensorflow_hub tensorflow_datasets

In [ ]:
# imports
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [ ]:
# load the dataset
(train_ds, val_ds), ds_info = tfds.load(
    'stanford_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

# number of classes
num_classes = ds_info.features['label'].num_classes

In [ ]:
# define image size and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# apply preprocessing, batching, and prefetching
train_ds = train_ds.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# ResNetV2-50 feature vector model
feature_extractor_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# feature extractor layer
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_url,
    input_shape=IMAGE_SIZE + (3,),
    trainable=False
)

In [ ]:
# build the model
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
# compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# train the model
EPOCHS = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

In [ ]:
# visualize the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

In [ ]:
# plot accuracy
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
# plot loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# evaluate model predictions
for images, labels in val_ds.take(1):
    predictions = model.predict(images)
    predicted_labels = tf.argmax(predictions, axis=1)
    plt.figure(figsize=(15, 15))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        true_label = ds_info.features['label'].int2str(labels[i].numpy())
        pred_label = ds_info.features['label'].int2str(predicted_labels[i].numpy())
        plt.title(f"True: {true_label}\nPred: {pred_label}")
        plt.axis("off")
    plt.show()

In [ ]:
# save the model
model.save('saved_model/stanford_dogs_resnetv2')

# load the model
reloaded_model = tf.keras.models.load_model('saved_model/stanford_dogs_resnetv2', custom_objects={'KerasLayer': hub.KerasLayer})